In [26]:
import requests
import pandas
import pandas as pd
from pprint import pprint
import quandl
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)


In [27]:
interestrate = quandl.get("RBA/F01_1", start_date="2010-01-01", end_date="2020-12-31")
#interestrate.tail()
month_by_month_cashrate = interestrate['Cash Rate Target. Units: Per cent; Series ID: FIRMMCRT']
month_by_month_cashrate_df = pd.DataFrame(month_by_month_cashrate)
month_by_month_cashrate_df.rename(columns = {'Cash Rate Target. Units: Per cent; Series ID: FIRMMCRT':'RBA_Rate'}, inplace = True)
#month_by_month_cashrate_df['InterestAmount']=0.0

money_borrowed = 100000

#for i, row in month_by_month_cashrate_df.iterrows():
    #interest_rate_per_month = month_by_month_cashrate_df['RBA_Rate']
    #interest_amount = (money_borrowed * interest_rate_per_month) / 1200
    #month_by_month_cashrate_df.at[i,'InterestAmount'] = interest_amount[i]
 
month_by_month_cashrate_df.tail(10)


,RBA_Rate
Date,
2020-03-31,0.43
2020-04-30,0.25
2020-05-31,0.25
2020-06-30,0.25
2020-07-31,0.25
2020-08-31,0.25
2020-09-30,0.25
2020-10-31,0.25
2020-11-30,0.11


In [23]:
month_by_month_cashrate_df.describe()


,RBA_Rate
count,132.000000
mean,2.391856
std,1.332457
min,0.100000
25%,1.500000
50%,2.000000
75%,3.329375
max,4.750000


In [28]:
def StartARIMAForecasting(Actual, P, D, Q):
	model = ARIMA(Actual, order=(P, D, Q))
	model_fit = model.fit(disp=0)
	prediction = model_fit.forecast()[0]
	return prediction

In [29]:
#predicted = StartARIMAForecasting(month_by_month_cashrate_df, 1,1,0)
#display the value
#print('Predicted=%f' % (predicted))

In [40]:
NumberOfElements = len(month_by_month_cashrate_df)

#Use 70% of data as training, rest 30% to Test model
TrainingSize = int(NumberOfElements * 0.7)
TrainingData = month_by_month_cashrate_df[0:TrainingSize]
TestData = month_by_month_cashrate_df[TrainingSize:NumberOfElements]

#new arrays to store actual and predictions
Actual = [x for x in TrainingData]
Predictions = list()


#in a for loop, predict values using ARIMA model
for timepoint in range(len(TestData)):
	ActualValue =  TestData[timepoint+1]
	#forcast value
	Prediction = StartARIMAForecasting(Actual, 3,1,0)    
	print('Actual=%f, Predicted=%f' % (ActualValue, Prediction))
	#add it in the list
	Predictions.append(Prediction)
	Actual.append(ActualValue)

#Print MSE to see how good the model is
#Error = MeanSquaredError(TestData, Predictions)
#print('Test Mean Squared Error (smaller the better fit): %.3f' % Error)
# plot
pyplot.plot(TestData)
pyplot.plot(Predictions, color='red')
pyplot.show()

KeyError: 1